## Lab 3 Part 2 - EECE.4860/5860 at UMass Lowell

Complete the missing code to execute a complex query that invokes search and calculation tools.

In [1]:
import sys
!{sys.executable} -m pip install --upgrade langchain langchain-community langchain_nvidia_ai_endpoints wikipedia jsonpointer numexpr


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11728 sha256=9acbc7e51f79938c4b997634986fd661fb99a2a32ca76b465500b35bb5ccde22
  Stored in directory: /home/u36e87926e39e265cdd3b4f969ee677d/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [1]:
import getpass
import os
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# You need to apply for a free API key from NVIDIA at https://build.nvidia.com
if not os.environ.get("NVIDIA_API_KEY"):
  os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter API key for NVIDIA: ")

llm = ChatNVIDIA(model="meta/llama-3.1-70b-instruct",
                           temperature=0.1,
                            max_tokens=512,
                            top_p=1.0,)

Enter API key for NVIDIA:  ········


In [2]:
from langchain.chains import LLMMathChain
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import Tool

wikipedia_wrapper = WikipediaAPIWrapper()
wikipedia_tool = Tool(
    name="Wikipedia",
    func=wikipedia_wrapper.run,
    description="A tool for searching the Internet to find various information on the topics mentioned."
)
math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
calculator = Tool.from_function(
    name="Calculator",
    func=math_chain.run,
    description="A tool for solving mathematical problems. Provide only the mathematical expressions."
)

# TODO: set up tools list
# tools=[...,...]
tools = [wikipedia_tool, calculator]

In [3]:
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

# TODO: create LangChain agent using initialize_agent()
agent = initialize_agent(
    # TODO: initiate tools list
    # tools=...,
    # TODO: initiate llm with chat model
    # llm=...,
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    handle_parsing_errors=True
)

print(agent.invoke(
    {"input": "In what year was the film Departed with Leopnardo Dicaprio released? What is this year raised to the 0.43 power?"}))

# TODO: try a different query
print(agent.invoke(
    {"input": "What is the capital of Sweden, and how many letters are in that word squared?"}))

/tmp/ipykernel_3386148/406131595.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new LLMMathChain chain...
2006^0.43```text
2006**0.43
```
...numexpr.evaluate("2006**0.43")...

Answer: 26.30281917656938
> Finished chain.
{'input': 'In what year was the film Departed with Leopnardo Dicaprio released? What is this year raised to the 0.43 power?', 'output': 'The film "The Departed" starring Leonardo DiCaprio was released in 2006. This year raised to the power of 0.43 is 26.30281917656938.'}


> Entering new LLMMathChain chain...
9^2```text
9**2
```
...numexpr.evaluate("9**2")...

Answer: 81
> Finished chain.
{'input': 'What is the capital of Sweden, and how many letters are in that word squared?', 'output': 'Stockholm, 81'}
